<a href="https://colab.research.google.com/github/mohammad-hosein/IMDB-SentimentAnalysis/blob/main/IMDB_SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import clear_output
!pip install --upgrade kaggle
clear_output()
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 71 bytes


In [ ]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

 78% 20.0M/25.7M [00:00<00:00, 85.8MB/s]
100% 25.7M/25.7M [00:00<00:00, 85.9MB/s]


In [ ]:
!unzip -qx /content/imdb-dataset-of-50k-movie-reviews.zip

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
data_path = '/content/IMDB Dataset.csv'
df = pd.read_csv(data_path)
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [ ]:
sentence = list(df['review'])
sentence[1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [ ]:
df['length'] = df.review.apply(lambda x: len(x.split()))
df

,review,sentiment,length
0,One of the other reviewers has mentioned that ...,positive,307
1,A wonderful little production. <br /><br />The...,positive,162
2,I thought this was a wonderful way to spend ti...,positive,166
3,Basically there's a family where a little boy ...,negative,138
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,230
...,...,...,...
49995,I thought this movie did a down right good job...,positive,194
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,112
49997,I am a Catholic taught in parochial elementary...,negative,230
49998,I'm going to have to disagree with the previou...,negative,212


In [ ]:
df['length'].describe()

count    50000.000000
mean       231.156940
std        171.343997
min          4.000000
25%        126.000000
50%        173.000000
75%        280.000000
max       2470.000000
Name: length, dtype: float64

In [ ]:
df = pd.get_dummies(df, columns=['sentiment'])
df = df.drop('sentiment_negative', 1)
df

,review,length,sentiment_positive
0,One of the other reviewers has mentioned that ...,307,1
1,A wonderful little production. <br /><br />The...,162,1
2,I thought this was a wonderful way to spend ti...,166,1
3,Basically there's a family where a little boy ...,138,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",230,1
...,...,...,...
49995,I thought this movie did a down right good job...,194,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",112,0
49997,I am a Catholic taught in parochial elementary...,230,0
49998,I'm going to have to disagree with the previou...,212,0


In [ ]:
sentences = list(df['review'])
labels = list(df['sentiment_positive'])
samples_size = len(df)

In [ ]:
sentences[1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [ ]:
labels[1]

1

In [ ]:
vocab_size = 50000
max_length = 250
oov_token = "<OOV>"

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = 50000, oov_token=oov_token)
tokenizer.fit_on_texts(sentences)

In [ ]:
sequences = tokenizer.texts_to_sequences(sentences)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded = pad_sequences(sequences, maxlen = max_length, padding='post', truncating='post')

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(padded, labels, test_size=0.2, random_state=101)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(40000, 250)
(10000, 250)
(40000,)
(10000,)


In [ ]:
embedding_dim = 100
from keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))
model.add(LSTM(64))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [ ]:
num_epochs = 15
history = model.fit(X_train, Y_train,
                    epochs = num_epochs, validation_data = (X_test,Y_test),
                    verbose = 1, batch_size = 128)

Epoch 1/15
313/313 [==============================] - 49s 78ms/step - loss: 0.6857 - accuracy: 0.5389 - val_loss: 0.6840 - val_accuracy: 0.5254
Epoch 2/15
313/313 [==============================] - 23s 75ms/step - loss: 0.6691 - accuracy: 0.5557 - val_loss: 0.6154 - val_accuracy: 0.6149
Epoch 3/15
313/313 [==============================] - 24s 76ms/step - loss: 0.6078 - accuracy: 0.6575 - val_loss: 0.6465 - val_accuracy: 0.6200
Epoch 4/15
313/313 [==============================] - 24s 76ms/step - loss: 0.6104 - accuracy: 0.6526 - val_loss: 0.5541 - val_accuracy: 0.7358
Epoch 5/15
313/313 [==============================] - 24s 75ms/step - loss: 0.5640 - accuracy: 0.6920 - val_loss: 0.4833 - val_accuracy: 0.8092
Epoch 6/15
313/313 [==============================] - 24s 76ms/step - loss: 0.5712 - accuracy: 0.6803 - val_loss: 0.6856 - val_accuracy: 0.5536
Epoch 7/15
313/313 [==============================] - 24s 76ms/step - loss: 0.4975 - accuracy: 0.7564 - val_loss: 0.4962 - val_accuracy:

In [ ]:
set1 = 'one of the best movies I ever seen in my life,I love that'
seq1 = tokenizer.texts_to_sequences([set1])
pad1 = pad_sequences(seq1, maxlen = max_length, padding='post', truncating='post')

In [ ]:
print(model.predict(pad1))

[[0.98837286]]
